Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [1]:
from collections import namedtuple
from random import choice
from tqdm.auto import tqdm
import numpy as np

In [2]:
import heapq

In [3]:
PUZZLE_DIM = 4
action = namedtuple('Action', ['pos1', 'pos2'])

In [4]:
def available_actions(state: np.ndarray) -> list['Action']:
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = list()
    if x > 0:
        actions.append(action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x, y + 1)))
    return actions



def do_action(state: np.ndarray, action: 'Action') -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state

In [5]:
RANDOMIZE_STEPS = 100_000
state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    state = do_action(state, choice(available_actions(state)))
initial_state = state.copy()
initial_state

Randomizing:   0%|          | 0/100000 [00:00<?, ?it/s]

Randomizing: 100%|██████████| 100000/100000 [00:00<00:00, 105614.91it/s]


array([[ 8, 13,  2,  9],
       [ 6, 14,  4,  0],
       [ 7, 10,  3, 12],
       [ 1, 15, 11,  5]])

In [6]:
MAX_STEP = 1_000_000

def heuristic(state, goal_state, n):
    """
    Combina la distanza di Manhattan e il numero di tessere fuori posizione.
    Combines the Manhattan distance and the number of tiles out of place.
    """
    manhattan = 0
    out_of_place = 0
    exp = 1.5  #Exponent applied to the Manhattan Distance plus the number of misplaced tiles.

    for x in range(n):
        for y in range(n):
            value = state[x, y]
            if value != 0:  # # Ignore the empty tile.
                # Calculation of Manhattan Distance
                target_x, target_y = divmod(np.argwhere(goal_state == value)[0][0], n)
                manhattan += abs(x - target_x) + abs(y - target_y)
                
                # Count the misplaced tiles
                if (x, y) != (target_x, target_y):
                    out_of_place += 1

    # Combine the two heuristics and raise the result to a power to make the heuristic more aggressive.
    h = (manhattan + out_of_place)**exp
    return h

'''def heuristic(state, goal, n):
    """
    Calcola l'euristica del Linear Conflict per un puzzle N*N.
    
    state: Matrice NumPy che rappresenta lo stato corrente del puzzle.
    goal: Matrice NumPy che rappresenta lo stato obiettivo.
    n: Dimensione del puzzle (es: 3 per un 3x3).
    """
    manhattan_distance = 0
    linear_conflict = 0
    exp = 1.2  # Esponente che eleva la distanza di Manhattan più il costo del conflitto lineare
    
    # Calcolo della Manhattan Distance e conflitti lineari per le righe
    for row in range(n):
        current_row = state[row, :]
        goal_row = goal[row, :]
        
        for i in range(n):
            if current_row[i] != 0 and current_row[i] in goal_row:
                # Calcola la Manhattan Distance per la tessera
                target_row, target_col = divmod(current_row[i] - 1, n)
                manhattan_distance += abs(row - target_row) + abs(i - target_col)
                
                # Controlla conflitti nella riga
                for j in range(i + 1, n):
                    if current_row[j] != 0 and current_row[j] in goal_row:
                        target_row_j, target_col_j = divmod(current_row[j] - 1, n)
                        if target_row == row and target_col > target_col_j:
                            linear_conflict += 2  # Aggiungi costo per il conflitto

    # Calcolo dei conflitti lineari per le colonne
    for col in range(n):
        current_col = state[:, col]
        goal_col = goal[:, col]
        
        for i in range(n):
            if current_col[i] != 0 and current_col[i] in goal_col:
                # Calcola la Manhattan Distance per la tessera
                target_row, target_col = divmod(current_col[i] - 1, n)
                
                # Controlla conflitti nella colonna
                for j in range(i + 1, n):
                    if current_col[j] != 0 and current_col[j] in goal_col:
                        target_row_j, target_col_j = divmod(current_col[j] - 1, n)
                        if target_col == col and target_row > target_row_j:
                            linear_conflict += 2  # Aggiungi costo per il conflitto
    
    #h = (manhattan_distance + linear_conflict) ** exp
    h = manhattan_distance ** exp
    return h'''

# Find the position of the 0 (empty tile).
def find_zero(state):
    return tuple(np.argwhere(state == 0)[0])

# Generate the valid successor states.
def get_neighbors(state, PUZZLE_DIM):
    neighbors = []
    x, y = find_zero(state)
    moves = {'up': (x - 1, y), 'down': (x + 1, y), 'left': (x, y - 1), 'right': (x, y + 1)}
    
    for move, (new_x, new_y) in moves.items():
        if 0 <= new_x < PUZZLE_DIM and 0 <= new_y < PUZZLE_DIM:  # Valid move
            new_state = state.copy()
            new_state[x, y], new_state[new_x, new_y] = new_state[new_x, new_y], new_state[x, y]  # Swap
            neighbors.append((new_state, move))
    return neighbors

# Reconstructs the path from the initial node to the goal node.
def reconstruct_path(came_from, current):
    path = []
    while current in came_from:
        current, move = came_from[current]
        path.append(move)
    return path[::-1]  # Reverse the path.

# A* algorithm
def a_star(initial_state, goal_state, PUZZLE_DIM):
    open_set = []
    heapq.heappush(open_set, (0, initial_state.tobytes()))  # (f_score, serialized state)
    
    came_from = {}
    g_score = {initial_state.tobytes(): 0}
    f_score = {initial_state.tobytes(): heuristic(initial_state, goal_state, PUZZLE_DIM)}
    step = 0  # Initialization of the step variable.


    while open_set and step < MAX_STEP: # Limit the number of steps.
    #while open_set: # Continue until a solution is found.
        _, current_bytes = heapq.heappop(open_set)
        current = np.frombuffer(current_bytes, dtype=int).reshape((PUZZLE_DIM, PUZZLE_DIM))

        # Check if it is the goal node.
        if np.array_equal(current, goal_state):
            return True, current, reconstruct_path(came_from, current_bytes), g_score[current_bytes]
        
        # Explore the neighbors.
        for neighbor, move in get_neighbors(current, PUZZLE_DIM):
            neighbor_bytes = neighbor.tobytes()
            tentative_g_score = g_score[current_bytes] + 1  # Cost of the move.
            
            if neighbor_bytes not in g_score or tentative_g_score < g_score[neighbor_bytes]:
                came_from[neighbor_bytes] = (current_bytes, move)
                g_score[neighbor_bytes] = tentative_g_score
                f_score[neighbor_bytes] = tentative_g_score + heuristic(neighbor, goal_state, PUZZLE_DIM)
                heapq.heappush(open_set, (f_score[neighbor_bytes], neighbor_bytes))
        step += 1  # Increment the step counter.
        
    #return current, None, None  # No solution found.
    return False, current, reconstruct_path(came_from, current_bytes), g_score[current_bytes]


In [7]:
goal_state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
goal_state

array([[ 1,  2,  3,  4],
       [ 5,  6,  7,  8],
       [ 9, 10, 11, 12],
       [13, 14, 15,  0]])

In [8]:
found, final_state, solution, cost = a_star(initial_state, goal_state, PUZZLE_DIM)
if found:
    print(f"Solution found in {cost} moves:")
    #print(" -> ".join(solution))
else:
    print(f"Solution not found. Best result in {cost} moves:")
print(" -> ".join(solution))
print(final_state)

Solution not found. Best result in 77 moves:
down -> down -> left -> left -> up -> right -> right -> down -> left -> left -> up -> left -> down -> right -> up -> right -> up -> left -> down -> right -> right -> up -> left -> up -> right -> down -> left -> left -> left -> down -> right -> up -> up -> left -> down -> down -> right -> up -> right -> down -> left -> down -> right -> up -> right -> down -> left -> left -> up -> up -> right -> right -> down -> down -> left -> left -> up -> left -> down -> right -> right -> up -> up -> left -> down -> down -> left -> up -> up -> right -> down -> right -> up -> right -> down -> down -> left
[[ 1  8  9 14]
 [ 6  7 15 13]
 [ 4  2 11 10]
 [ 3  5  0 12]]
